In [1]:
import os 
import re
import requests
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from datetime import date
from datetime import datetime

from metapub.convert import doi2pmid
from metapub import PubMedFetcher

import pandas as pd
import numpy as np
from lxml import etree
from Bio import Entrez

2023-02-19 18:49:05 herman-login3.hpc.lji.org metapub.config[16877] WARNING NCBI_API_KEY was not set.


## Query the NCBI database

In [2]:
# use one of the following search queries/filters
# filters = "HiChIP[All Fields] AND \"gse\"[Filter]" # the whole notebook will take around 7 min to run
# filters = "HiChIP[All Fields] AND (\"gse\"[Filter] AND (\"Homo sapiens\"[Organism] OR \"Mus musculus\"[Organism]))
# filters = "HiChIP[All Fields] AND (\"Homo sapiens\"[Organism] AND \"gse\"[Filter])"
# filters = "HiChIP[All Fields] AND (\"Homo sapiens\"[Organism] AND \"published last year\"[Filter]) AND \"gse\"[Filter]" # 2 min
# filters = "HiChIP[All Fields] AND (\"Mus musculus\"[Organism] AND \"gse\"[Filter])"

# set a dummy email 
Entrez.email = "zjiang@lji.org"

# query the NCBI database 
filters = "HiChIP[All Fields] AND (\"gse\"[Filter] AND (\"Homo sapiens\"[Organism] OR \"Mus musculus\"[Organism]))"
search_result = Entrez.esearch(db="gds", retmax=10000, term=filters)
ncbi_results = Entrez.read(search_result)

In [3]:
# get summary of this entry on GEO datasets (abbreviated as GDS)
ncbi_ids = ','.join(ncbi_results['IdList'])
geo_query_handle = Entrez.esummary(db="gds", id=ncbi_ids, retmode="xml") 
geo_entries = list(Entrez.parse(geo_query_handle))

## Cleaning the Paper Names and Converting from PMID to DOI

Extract PMIDs

In [4]:
pubmed_ids = set()
for entry in geo_entries:
    for value in entry['PubMedIds']:
        pubmed_ids.add(str(int(value)))
pubmed_ids = sorted(pubmed_ids)
pubmed_ids[0:10]

['27643841',
 '28945252',
 '29224777',
 '29641996',
 '29706538',
 '29731168',
 '30046115',
 '30122536',
 '30397335',
 '30580963']

Convert from PMID to DOI using the eutils API

In [5]:
# Define the base URL for the efetch command
efetch_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'

# define the parameters for the efetch command
params = {
    'db': 'pubmed',
    'id': ','.join(pubmed_ids),
    'retmode': 'xml',
    'rettype': 'docsum'
}

# send the request to the E-utilities API and get the response
ds_response = requests.get(efetch_url, params=params)

# parse the XML response and extract the DOIs
ds_root = etree.fromstring(ds_response.content)
doc_sums = ds_root.findall('DocSum')
pubmed_data = []
for doc_sum in doc_sums:

    pmid = doc_sum.find('Id').text
    dois = [x.text for x in doc_sum.findall('Item[@Name="DOI"]')]
    dois = ','.join(dois)
    pubmed_data.append([pmid, dois])

pubmed_data = pd.DataFrame(pubmed_data, columns=['PMID', 'DOI'])
pubmed_data.set_index('PMID', inplace=True)
pubmed_data = pubmed_data.squeeze()

#### Convert GEO entries into a dataframe

In [8]:
geo_data = []
for i, entry in enumerate(geo_entries):

    ##### Cleaning up the title #####
    paper_title = entry['title']
    new_paper_title = ''
    last_word = re.split('\s|[.]', paper_title)[-1]
    last_two_words = re.split('\s|[.]', paper_title)[-2:]
    
    # remove ".[HiChIP]", ". [HiChIP]", or "(HiChIP)" in paper titles using steps below
    if ('[' in last_word and ']' in last_word) or \
            ('(' in last_word and ')' in last_word):
        new_paper_title = re.split('\s|[.]', paper_title)[0:-1]
        new_paper_title = ' '.join(new_paper_title)
        new_paper_title = new_paper_title.strip()

        # remove the last period in paper title
        if new_paper_title.endswith('.'):
            new_paper_title = new_paper_title[:-1]
            
    # remove ".[Hi ChIP]", ". [Hi ChIP]", or "[Bead Array]" in paper titles using steps below
    elif ('[' in last_two_words[0] and ']' in last_two_words[1]) or \
                ('(' in last_two_words[0] and ')' in last_two_words[1]):
        new_paper_title = re.split('\s|[.]', paper_title)[0:-2]
        new_paper_title = ' '.join(new_paper_title)
        new_paper_title = new_paper_title.strip()

        # remove the last period in paper title
        if new_paper_title.endswith('.'):
            new_paper_title = new_paper_title[:-1]
            
    # remove the period
    elif paper_title.endswith('.'):
        new_paper_title = paper_title[:-1] 
        
    else:
        new_paper_title = paper_title

    new_paper_title = new_paper_title.replace("\xa0", " ") # remove no-break space
    entry['title'] = new_paper_title

    ##### add the doi to the entry #####
    curr_dois = []
    for value in entry['PubMedIds']:
        pubmed_id = str(int(value))
        curr_doi = pubmed_data[pubmed_id]
        curr_dois.append(curr_doi)
    entry['DOI'] = ','.join(curr_dois)

    ##### add the adjusted entry to the data #####
    geo_data.append(entry)
    
geo_df = pd.DataFrame(geo_data)

In [9]:
geo_df.head()

,Item,Id,Accession,GDS,title,summary,GPL,GSE,taxon,entryType,...,n_samples,SeriesTitle,PlatformTitle,PlatformTaxa,SamplesTaxa,PubMedIds,Projects,FTPLink,GEO2R,DOI
0,[],200223259,GSE223259,,Chromatin accessibilities in C9HRE ALS/FTD pat...,This SuperSeries is composed of the SubSeries ...,16791,223259,Homo sapiens,GSE,...,24,,,,,[],[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE223nn...,yes,
1,[],200223258,GSE223258,,To profile the chromatin structural changes re...,DAXX is an epigenetic regulator and chromatin ...,16791,223258,Homo sapiens,GSE,...,3,,,,,[],[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE223nn...,yes,
2,[],200178673,GSE178673,,Chromatin landscape in perinatal cardiomyocytes,In order to understand the transcriptional reg...,21273;24688,178673,Mus musculus; Rattus norvegicus,GSE,...,28,,,,,"[IntegerElement(36653336, attributes={})]",[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE178nn...,yes,10.1038/s41420-023-01322-3
3,[],200215020,GSE215020,,Generate 3D genome database for atrial and ven...,Measurement the contribution of 3D genome stru...,24247,215020,Mus musculus,GSE,...,6,,,,,[],[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE215nn...,yes,
4,[],200215065,GSE215065,,Regulation of mouse chamber selective enhancers,This SuperSeries is composed of the SubSeries ...,19057;26526;24247,215065,synthetic construct; Mus musculus,GSE,...,82,,,,,"[IntegerElement(36705030, attributes={})]",[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE215nn...,yes,10.1161/CIRCULATIONAHA.122.061955


## Reformat the Columns for Google Sheet Compatibility

In [10]:
# drop columns
drop_cols = ['Item', 'Id', 'GDS', 'GPL', 'GSE', 'entryType', 'ptechType',
             'valType', 'SSInfo', 'subsetInfo', 'suppFile', 'Relations', 'ExtRelations',
             'n_samples', 'SeriesTitle', 'PlatformTitle', 'PlatformTaxa', 'SamplesTaxa',
             'Projects', 'FTPLink', 'GEO2R']
geo_df.drop(drop_cols, inplace=True, axis=1)

# rename columns
rename_cols = {"Accession":"GEO / Data link",
               "title": "Paper Title",
               "taxon": "Organism",
               "gdsType": "Any other information",
               "PDAT": "Year",
               "Samples": "Other matched data"}
geo_df.rename(columns=rename_cols, inplace=True)

# extract just the year
geo_df["Year"] = geo_df["Year"].str[:4]

# add index for merging
geo_df['index'] = np.arange(len(geo_df))

# fill na with empty values
geo_df.fillna("", inplace=True)

# adding missing columns with empty values
geo_df["Journal"] = ""
geo_df["Authors"] = ""
geo_df["Tissue/Cell Line"] = ""
geo_df["Presenter"] = ""
geo_df["Potential HiChIP"] = ""

## Assigning Potential HiChIP Samples

Filter out samples with no HiChIP mention and assigned Yes or Maybe to all others 

In [13]:
lst = []
GSM_IDs = []
index = 0
for row in geo_df["Other matched data"]:
    
    temp = []
    state = False
    
    # Check for the term HiChIP in the title
    for ele in row: 
        if ("HiChIP".casefold() in ele["Title"].casefold()) or \
                ("Hi-ChIP".casefold() in ele["Title"].casefold()):
            state = True
    
    # keep samples with HiChIP and mark "Yes"
    if state == True: 
        for ele in row:
            GSM_IDs.append(ele["Accession"])
            if ("HiChIP".casefold() in ele["Title"].casefold()) or \
                    ("Hi-ChIP".casefold() in ele["Title"].casefold()):
                temp.append(ele["Accession"] + ": " + ele["Title"])
        temp_str = "\n".join(temp)
        geo_df.at[index, "Potential HiChIP"] = "Yes"

    # otherwise, keep all GSM samples and mark "Maybe"
    else: 
        for ele in row:
            GSM_IDs.append(ele["Accession"])
            temp.append(ele["Accession"] + ": " + ele["Title"])
        temp_str = "\n".join(temp)
        geo_df.at[index,"Potential HiChIP"]="Maybe"
    lst.append(temp_str)
    index += 1
    
geo_df["Other matched data"] = list(lst)

## Merge Rows with the Same Paper Title

In [14]:
agg_functions = {'Paper Title':'first',
                    'DOI':'max',
                    'Journal':'first',
                    'Authors':'first',
                    'Year':'first',
                    'GEO / Data link': lambda x: '\n'.join(x),
                    'Any other information':lambda x: '\n'.join(x),
                    'Organism':'first', 'Tissue/Cell Line':'first',
                    'Potential HiChIP':'first',
                    'Other matched data':lambda x: '\n'.join(x),
                    'Presenter':'first'}
geo_df_grouped = geo_df.groupby(geo_df['Paper Title']).aggregate(agg_functions)

## Fetch Journal Name and First Author of Each Paper and Add https to DOI

In [15]:
### JR: Extremely slow, need to find a better way to do this

# jounrnal_lst = []
# authors_lst = []
# fetch = PubMedFetcher()
# for DOI in geo_df_grouped['DOI']:
#     try:
#         PMID = doi2pmid(DOI)
#         article = fetch.article_by_pmid(PMID)
#         jounrnal_lst.append(article.journal)
#         authors_lst.append(article.authors[0].split()[0] + " et al.")
#     except:
#         jounrnal_lst.append("")
#         authors_lst.append("")
# geo_df_grouped['Journal'] = jounrnal_lst
# geo_df_grouped['Authors'] = authors_lst

In [112]:
# # add https address to DOI (for easy access)
# geo_df_grouped.loc[(geo_df_grouped['DOI'] != ''), 'DOI'] = 'https://doi.org/' + geo_df_grouped.loc[(geo_df_grouped['DOI'] != ''), 'DOI']

## Note the Date Added, Set Output Path, and Save the Final Output File

In [113]:
# make an output directory
######## new for cell type
outdir = ''
# outdir = 'results/hichip_db/'
# os.makedirs(outdir, exist_ok=True)

In [114]:
# determining the current year, month and day
today = date.today()
date_str = today.strftime("%Y_%m_%d")

# determining current hour and minute
now = datetime.now()
time_str = now.strftime("%H_%M")

# setting the output filename
output = os.path.join(outdir, "GEO_Query.{}_{}".format(date_str, time_str))

In [115]:
# add a column that notes the added date
geo_df_grouped['Date Added'] = date_str.replace('_', '-')
reorder = ['Paper Title', 'Journal', 'Authors', 'Year', 'DOI', 'GEO / Data link', 'Any other information',
           'Organism', 'Tissue/Cell Line', 'Potential HiChIP', 'Other matched data', 'Presenter', 'Date Added']
geo_df_grouped = geo_df_grouped.loc[:, reorder]

In [116]:
geo_df_grouped.to_excel(output + ".xlsx", index=False)

## Save the GSM IDs line by line

In [117]:
GSM_IDs = list(set(GSM_IDs))
with open(r'gsm_list.txt', 'w') as fp:
    for gsm in GSM_IDs:
        # write each item on a new line
        fp.write("%s\n" % gsm)

## Query GEO BioSample using the GSM IDs

In [118]:
GSM_IDs_from_file = []

# open file and read the content in a list
with open(r'gsm_list.txt', 'r') as fp:
    for line in fp:
        GSM_IDs_from_file.append(line)

GSM_filters = ' [All Fields] OR '.join(GSM_IDs_from_file[4000:])


search_result = Entrez.esearch(db="biosample", retmax=10000, term=GSM_filters)
result = Entrez.read(search_result)

### Filter query result by looping through XML

#### Dict of organs and their synynoms for classification

In [120]:
json = OrderedDict()
# order the classifcations from general to specific, so that as the code goes
# down the ordered dict, general classifcations are replaced with specific ones
json = {
    "biomaterial": {
        "cell line": ["cellline", "cell line", "soxgfp", "cell line established from", "hESC"], # pay attention to the last two
        "primary": ["primary", "primi", "attribute_name=\"patient\""]
    },
    "disease": {
        "HCC": [">hcc<"],
        "GIST": ["gist"],
        "Colorectal Adenocarcinoma": ["cw2", ">rko", "sw403", "sw480", "sw620", "sw837", "sw948", "dld1", "hct15", "snu-c5", "hcc116", "ls174t", "colon cancer", "colorectal cancer", "coloO320-dm"],
        "Leukemia": ["leukemia"],
        "Acute myeloid leukemia": ["mll-af9"],
        "Acute monocytic leukemia": ["thp-1"],
        "Acute lymphoblastic leukaemia": [">sem"],
        "Mantle cell lymphoma": ["mantle cell lymphoma"],
        "ALL": [">all<"],
        "Burkitt's lymphoma": ["ramos b"],
        "Non-Hodgkin lymphoma": ["oci-ly7", "sudh4"],
        "Prostate carcinoma": ["prostate cancer", "lncap", "vcap"],
        "Normal": [">fhc"],
        "Gastrointestinal stromal tumour": ["gist"],
        "Mycosis fungoides": ["my-la", "myla"],
        "Breast cancer": ["breast cancer", "t47d"],
        "Invasive Breast Carcinoma": ["hcc1599"],
        "Triple-negative breast cancer": ["triple-negative breast cancer"],
        "ER-positive, PR-positive, HER2-negative breast cancer": ["mcf7"],
        "ER-positive breast cancer": ["mda_mb134vi"],
        "ER-positive, PR-negative and ERBB2-negative breast cancer": ["sum44pe"],
        "Esophageal squamous cancer": ["esophageal squamous cancer"],
        "Non-Small Cell Lung Cancer": [">hcc15<"],
        "Lung squamous cancer": ["lung squamous cancer"],
        "Small cell lung cancer": ["small cell lung cancer"],
        "Head and neck squamous cancer": ["head and neck squamous cancer"],
        "Human endometrial adenocarcinoma": ["human endometrial adenocarcinoma"],
        "Ewing sarcoma": ["ewing sarcoma"],
        "Multiple myeloma": ["multiple myeloma"],
        "Neuroblastoma": ["neuroblastoma", "tr-14", "tr14"],
        "Gastric carcinoma": ["snu16", "snu-16"],
        "Rhabdoid tumor": ["g401"],
        "Alveolar rhabdomyosarcoma": [">rh4<", ">rh3<", ">rh30<"],
        "Embryonal rhabdomyosarcoma": [">sms"],
    },
    # QUESTION: what do I do with ARMS, which has RH3, 30, 4, 41 metasized to different organs
     # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3713458/?
    "organ": {
        # ewing sarcoma are cells in the bones or connective tissue # plb might cause problem
        "embryo": ["embryo", "mesc", "hesc", "\">e14", "\">esc"],
        "skeletal system": ["bone", "marrow", "ewing sarcoma", "t-all", "myeloma", "plb", "tc71", "\">rh30", "pre-b"],
        "blood": ["vessel", "leukemia", "blood", "ramos b", "haematopoietic", "b-all", "\">sem", "jurkat", "cutll1", "gm12878", "oci-ly7", "thp-1", "hudep1", "hudep-1", "hudep2", "hudep-2", "hudep3", "hudep-3"], # GSM6206927 bone or blood? # Hudep is from umbellical cord blood
        "bodily fluid": ["pleural", "effusion", "sudh4"],
        "skin": ["skin", "epi", "squamous", "breast", "melanoma", "erythematosus", "sle", "keratinocyte", "melanocyte", "nasopharyngeal"], # sle might cause problem
        "mammary gland": ["mammar", "breast", "mda-mb-231"],
        "thyroid": ["thyroid gland", "thyroid"],
        "thymus": ["thymus", "thymic", "treg", "t-reg", "my-la", "myla"], # GSM5680696 thyroid or epithelium
        "lymph node": ["lymph node", "lymphoma", "\">rh3\""], # RH3 is derived from metastatic site, lymph node
        "tonsil": ["tonsil"],
        "liver": ["liver", "hcc"],
        "musculature of body": ["muscula", "soft", "connective", "rhabdomyosarcoma", "angiomatoid fibrous histiocytoma", "\">sms", "a673"],
        "lung": ["lung", "\">rh4"],
        "esophagus": ["esophagus", "esophageal"], # GSM5680726 esophagus or epithelium
        "intestine": ["indestin", "fhc", "sw480"],
        "kidney": ["kidney", "podocyte", "g401"],
        "colon": ["colon", "colorectal", "hct116", "ls174t", ">rko", "coloO320-dm"],
        "spleen": ["spleen"],
        "stomach": ["stomach", "gist", "snu16", "snu-16"],
        "eye": ["retina"],
        "placenta": ["placenta"],
        "pancreas": ["pancrea"],
        "bladder": ["bladder"],
        "uturus": ["umbilical", "uterus", "endometrial", "cervic", "hela"],
        "penis": ["penis"],
        "testis": ["test"],
        "ovary": ["ovar"],
        "nerve": ["neuroblastoma", "nerve", "spinal", "cortex", "\">npc", "tr-14", "tr14"],
        "brain": ["brain", "caudate", "chla-10"],
        "prostate": ["prostate"],
        "heart": ["heart", "cardi", "ventricle", "atrium", "aortic"],
        # dupliate the classifications except the synonyms are now cell line names or cell type names?
    }
}

In [ ]:
# 50 s per 100 GSMs
# Initialize an empty dataframe with the desired column names
cell_type_df = pd.DataFrame(columns=['name',
                                     'organism',
                                     'biomaterial',
                                     'disease',
                                     'organ',
                                     'tissue',
                                     'celltype',
                                     'strain',
                                     'sex',
                                     'age',
                                     'extdb_name',
                                     'extdb_uuid',
                                     'other'])



In [191]:
new_list = sorted(set(result["IdList"]))

# Define the base URL for the efetch command
base_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi'

root = None
intervals = 100
for start in range(0, len(new_list), intervals):

    end = start + intervals
    print(start, end)

    # define the parameters for the efetch command
    params = {
        'db': 'biosample',
        'id': ','.join(new_list[start:end]),
        'retmode': 'xml',
        'rettype': 'DocumentSummarySet'
    }

    # send the request to the E-utilities API and get the response
    response = requests.get(base_url, params=params)

    if root == None:
        root = etree.fromstring(response.content)
    else:
        root2 = etree.fromstring(response.content)
        root.extend(root2)


0 100
100 200
200 300
300 400
400 500
500 600
600 700
700 800
800 900
900 1000
1000 1100
1100 1200
1200 1300
1300 1400
1400 1500
1500 1600
1600 1700
1700 1800
1800 1900
1900 2000
2000 2100
2100 2200
2200 2300
2300 2400
2400 2500
2500 2600
2600 2700
2700 2800
2800 2900
2900 3000
3000 3100
3100 3200
3200 3300
3300 3400
3400 3500
3500 3600
3600 3700
3700 3800
3800 3900
3900 4000
4000 4100
4100 4200
4200 4300
4300 4400
4400 4500
4500 4600
4600 4700
4700 4800
4800 4900
4900 5000
5000 5100
5100 5200
5200 5300


In [192]:

num_of_gsm_done = 0

# Iterate through the list of GSM IDs
for doc_sum in root.findall(".//DocumentSummary"):

    data = doc_sum.find('.//SampleData')

    gsm_data = {
        "name":"",
        "organism":"",
        "biomaterial":"",
        "disease": "",
        "organ":"",
        "tissue":"",
        "celltype":"",
        "strain":"",
        "sex":"",
        "age":"",
        "extdb_name":"",
        "extdb_uuid":"",
        "other":""
        }
    celltype_infos = []
    
    # Classify organ and biomaterial by checking synonyms in the SampleData XML text
    # check GSM4956217 and GSM4447223
    data_str = data.text
    for dic in json:
        for classification in json[dic]:
            synonyms = json[dic][classification]
            found = any(syn in data_str.lower() for syn in synonyms)
            # edge cases
            if "\">RD" in data_str:
                gsm_data["organ"] = "musculature of body"
            if "RH3" in data_str:
                gsm_data["organ"] = "exocrine gland"
            if classification == "liver": # sometimes hcc is actually breast, like hcc1599 and hcc19** cell lines
                if "breast" in data_str.lower():
                    gsm_data["organ"] = "breast"
            if found:
                gsm_data[dic] = "|" + classification
        gsm_data[dic] = gsm_data[dic].lstrip("|")
    
    
    for elem in data:
        for subelem in elem:
            if ('db\': \'GEO' in str(subelem.attrib)):
                gsm_data["name"] = subelem.text
            if "taxonomy_name" in str(subelem.attrib):
                gsm_data["organism"] = subelem.attrib["taxonomy_name"]
            if ('disease' in str(subelem.attrib) or 'cancer type' in str(subelem.attrib)):
                gsm_data["disease"] = subelem.text # many organ info are from here
            try:
                if (gsm_data["disease"] == "" and \
                    ("oma" in str(myroot.find("./Attributes/Attribute[@attribute_name='tissue']").text).lower() or \
                    "tumor" in str(myroot.find("./Attributes/Attribute[@attribute_name='tissue']").text).lower())
                   ):
                    gsm_data["disease"] = myroot.find("./Attributes/Attribute[@attribute_name='tissue']").text
            except:
                gsm_data["disease"] = gsm_data["disease"] + ""
            if (gsm_data["disease"] == "" and \
                ("tumor" in str(myroot.find("./Attributes/Attribute[@attribute_name='source_name']").text).lower() or \
                "tumour" in str(myroot.find("./Attributes/Attribute[@attribute_name='source_name']").text).lower() or \
                "normal" in str(myroot.find("./Attributes/Attribute[@attribute_name='source_name']").text).lower())
               ):
                    gsm_data["disease"] = myroot.find("./Attributes/Attribute[@attribute_name='source_name']").text
            if ('organ' in str(subelem.attrib)): # no organ attributes at all
                gsm_data["organ"] = subelem.text
            if ('tissue' in str(subelem.attrib) or \
                'brain region' in str(subelem.attrib)
               ):
                gsm_data["tissue"] = subelem.text  # most organ info are from here # can have disease info
            if ('cell_line' in str(subelem.attrib) or \
                'biological sample' in str(subelem.attrib) or \
                'neuronal subtype' in str(subelem.attrib) or \
                '\'cell' in str(subelem.attrib)
               ):
                if ((not str(subelem.text).startswith("BRD")) and (not str(subelem.text).startswith("MGH"))): # these are gene names
                    celltype_infos.append(str(subelem.attrib['attribute_name']) + "-->" + subelem.text)
            if ('parental cell line' in str(subelem.attrib)):
                celltype_infos.append(str(subelem.attrib['attribute_name']) + "-->" + subelem.text)
                gsm_data["biomaterial"] = "cell line"
            if ("mll-" in str(myroot.find("./Attributes/Attribute[@attribute_name='source_name']").text).lower() or \
                "bl6" in str(myroot.find("./Attributes/Attribute[@attribute_name='source_name']").text).lower() or \
                "hct" in str(myroot.find("./Attributes/Attribute[@attribute_name='source_name']").text).lower() or \
                "vcap" in str(myroot.find("./Attributes/Attribute[@attribute_name='source_name']").text).lower() or \
                "h9" in str(myroot.find("./Attributes/Attribute[@attribute_name='source_name']").text).lower() or \
                "cell line" in str(myroot.find("./Attributes/Attribute[@attribute_name='source_name']").text).lower() or \
                ("rim" not in str(myroot.find("./Attributes/Attribute[@attribute_name='source_name']").text).lower() and \
                 "cyte" in str(myroot.find("./Attributes/Attribute[@attribute_name='source_name']").text).lower())
               ):
                celltype_infos.append("{source_name}" + "-->" + str(myroot.find("./Attributes/Attribute[@attribute_name='source_name']").text))
                gsm_data["biomaterial"] = "cell line"
            if ('cell_line' in str(subelem.attrib)):
                if (subelem.text != "--"):
                    gsm_data["biomaterial"] = "cell line"
            if ('strain' in str(subelem.attrib) or 'mouse model' in str(subelem.attrib)):
                gsm_data["strain"] = subelem.text
            if ('p14' in str(myroot.find("./Attributes/Attribute[@attribute_name='source_name']").text).lower()):
                gsm_data["strain"] = str(myroot.find("./Attributes/Attribute[@attribute_name='source_name']").text)
            if ('gender' in str(subelem.attrib) or \
                '\'sex\'' in str(subelem.attrib)):
                gsm_data["sex"] = subelem.text
            if ("day" in str(myroot.find("./Attributes/Attribute[@attribute_name='source_name']").text).lower()):
                gsm_data["age"] = str(myroot.find("./Attributes/Attribute[@attribute_name='source_name']").text)
            if ('\'age\'' in str(subelem.attrib) or \
                'age_day' in str(subelem.attrib)
               ):
                gsm_data["age"] = subelem.text
            if ('\'time' in str(subelem.attrib) or \
                '\'differentiation' in str(subelem.attrib) or \
                'differentiation\'' in str(subelem.attrib) or \
                'developmental stage' in str(subelem.attrib) or \
                ('erythropoiesis' in str(subelem.attrib) and 'no' not in str(subelem.attrib))
               ):
                if gsm_data["age"] == "":
                    gsm_data["age"] = subelem.text
            
    gsm_data["celltype"] = '|'.join(set(celltype_infos))
    gsm_data["other"] = "{source_name}" + "-->" + str(myroot.find("./Attributes/Attribute[@attribute_name='source_name']").text)
    
    cell_type_df = cell_type_df.append(gsm_data, ignore_index=True)
    
    # indicate how many GSM are processed and save the table as the loop runs
    num_of_gsm_done += 1
    if (num_of_gsm_done % 100 == 0):
            print("finished GSM number", num_of_gsm_done)

finished GSM number 2600
finished GSM number 2700
finished GSM number 2800
finished GSM number 2900
finished GSM number 3000
finished GSM number 3100
finished GSM number 3200
finished GSM number 3300
finished GSM number 3400
finished GSM number 3500
finished GSM number 3600
finished GSM number 3700
finished GSM number 3800
finished GSM number 3900
finished GSM number 4000
finished GSM number 4100
finished GSM number 4200
finished GSM number 4300
finished GSM number 4400
finished GSM number 4500
finished GSM number 4600
finished GSM number 4700
finished GSM number 4800
finished GSM number 4900
finished GSM number 5000
finished GSM number 5100
finished GSM number 5200
finished GSM number 5300
finished GSM number 5400
finished GSM number 5500
finished GSM number 5600
finished GSM number 5700
finished GSM number 5800
finished GSM number 5900
finished GSM number 6000
finished GSM number 6100
finished GSM number 6200
finished GSM number 6300
finished GSM number 6400
finished GSM number 6500


### Save

In [ ]:
# setting the output filename
today = date.today()
date_str = today.strftime("%Y_%m_%d")
now = datetime.now()
time_str = now.strftime("%H_%M")
output = "GEO_Query_cell_type.{}_{}".format(date_str, time_str)
print("output file: ", output)
cell_type_df.to_excel(output + ".xlsx", index=False)

output file:  GEO_Query_cell_type.2022_12_23_18_16


Sample ID: This is the unique identifier for each sample in the database. You can search for samples by their ID using this field.

Organism: This field allows you to search for samples from a specific organism, such as human, mouse, or yeast.

Tissue: This field allows you to search for samples based on their tissue type, such as liver, brain, or heart.

Disease state: This field allows you to search for samples based on their disease state, such as cancer, diabetes, or autoimmune disease.

Cell type: This field allows you to search for samples based on their cell type, such as fibroblasts, macrophages, or stem cells.

Developmental stage: This field allows you to search for samples based on their developmental stage, such as embryonic, fetal, or adult.

Experimental design: This field allows you to search for samples based on the type of experiment they were used in, such as microarray, RNA-seq, or ChIP-seq.